In [ ]:
# This script attempts to find the best hgihway system for the US given certain cities and their population sizes.

In [208]:
from bs4 import BeautifulSoup
from collections import deque
from IPython.display import display

import copy
import folium
import heapq
import math
import numpy as np
import requests
import time

In [209]:
request = requests.get("https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population")
soup = BeautifulSoup(request.text)
city_pop_lat_long_list = []

for tr in soup.find_all("table", class_="wikitable")[1].find_all("tr"):
    tds = tr.find_all("td")
    if len(tds) == 0:
        continue
    new_city_pop_lat_long = [tds[0].text[:-1].split('[')[0], int("".join(tds[2].text[:-1].split(',')))]
    lat_long_strs = tds[-1].text.split('\ufeff')[2].strip().split()
    for coord_str in lat_long_strs:
        new_city_pop_lat_long.append(float(coord_str[:-2]))
    if new_city_pop_lat_long[0] == "Honolulu" or new_city_pop_lat_long[0] == "Anchorage":
        continue
    city_pop_lat_long_list.append(new_city_pop_lat_long)

In [242]:
class City:
    def __init__(self, name, population, lat, long):
        self.name = name
        self.population = population
        self.lat_long = np.array([lat, long])
        self.roads = set()

    def __lt__(self, other):
        if isinstance(other, City):
            return self.population < other.population
        return NotImplemented

    def __str__(self):
        return self.name

class Road:
    def __init__(self, city1, city2):
        if city1 == city2:
            print("Tried to connect the same city!")
        self.connected_cities = (city1, city2)
        self.distance = math.sqrt(sum((city1.lat_long - city2.lat_long)**2))
        self.connected_population = city1.population + city2.population
        city1.roads.add(self)
        city2.roads.add(self)

    def __lt__(self, other):
        if isinstance(other, Road):
            return self.distance < other.distance
        return NotImplemented

    def __str__(self):
        return "({0}, {1})".format(self.connected_cities[0].name, self.connected_cities[1].name)

In [275]:
all_cities = {}
all_roads = {}
for city in city_pop_lat_long_list:
    # construct all city objects
    all_cities["{0} ({1}, {2})".format(city[0], city[2], city[3])] = City(city[0], city[1], city[2], city[3])

city_keys = sorted(all_cities)
for i in range(len(city_keys)):
    for j in range(i+1, len(city_keys)):
        # construct all road objects
        # connect all cities
        all_roads[(city_keys[i], city_keys[j])] = Road(all_cities[city_keys[i]], all_cities[city_keys[j]])

shortest_paths = {}
for path in all_roads:
    # calculate all current shortest paths. Right now these are just the direct routes
    shortest_paths[path] = set([all_roads[path]])

In [276]:
roads_sorted_by_population = sorted(all_roads.values(), key=lambda road: road.connected_population)

In [277]:
i = 0
for road in roads_sorted_by_population[i:]:
    if i % 100 == 0:
        start_time = time.time()
    # temporarily remove road to city to find new optimal path
    city1, city2 = road.connected_cities
    if road not in city1.roads:
        continue
    city1.roads.remove(road)
    city2.roads.remove(road)

    new_shortest_path = {}
    road_cannot_be_replaced = False
    for path in shortest_paths:
        if road not in shortest_paths[path]:
            continue
        # max acceptable distance between connected cities in new graph
        max_new_dist = all_roads[path].distance*(1.4 - 0.3*math.log(all_roads[path].connected_population)/math.log(12078949))

        start_city, end_city = all_cities[path[0]], all_cities[path[1]]
        found_short_path = False
        explored_cities = {start_city: 0}
        frontier_heap = [(0, start_city, [])]
        while len(frontier_heap) != 0:
            current_distance, current_city, current_path = heapq.heappop(frontier_heap)
            if current_city == end_city:
                found_short_path = True
                new_shortest_path[path] = set(current_path)
                break

            if current_city in explored_cities and current_distance > explored_cities[current_city]:
                continue

            for next_road in current_city.roads:
                new_distance = current_distance + next_road.distance
                if new_distance > max_new_dist:
                    continue
                other_city = next_road.connected_cities[0] if current_city != next_road.connected_cities[0] else next_road.connected_cities[1] 
                if other_city in explored_cities and explored_cities[other_city] < new_distance:
                    continue
                explored_cities[other_city] = new_distance
                heapq.heappush(frontier_heap, (new_distance, other_city, current_path + [next_road]))

        if not found_short_path:
            road_cannot_be_replaced = True
            break

    if road_cannot_be_replaced:
        city1.roads.add(road)
        city2.roads.add(road)
    else:
        for path in new_shortest_path:
            shortest_paths[path] = new_shortest_path[path]

    if (i + 1) % 100 == 0:
        end_time = time.time()
        print(i + 1, "time: " + str(end_time - start_time))
    i += 1

100 time: 1.497070074081421
200 time: 1.4918580055236816
300 time: 1.3058559894561768
400 time: 1.2810781002044678
500 time: 1.356093168258667
600 time: 1.691880226135254
700 time: 1.4264609813690186
800 time: 1.4346930980682373
900 time: 1.3451170921325684
1000 time: 1.494385004043579
1100 time: 1.332655906677246
1200 time: 1.5027592182159424
1300 time: 1.4228992462158203
1400 time: 1.2882041931152344
1500 time: 1.440011739730835
1600 time: 1.4008839130401611
1700 time: 1.411221981048584
1800 time: 1.3793931007385254
1900 time: 1.3113782405853271
2000 time: 1.1988770961761475
2100 time: 1.2133498191833496
2200 time: 1.3889927864074707
2300 time: 1.1743359565734863
2400 time: 1.0904781818389893
2500 time: 1.3044531345367432
2600 time: 1.40535306930542
2700 time: 1.2514011859893799
2800 time: 1.3439507484436035
2900 time: 1.1719911098480225
3000 time: 1.2686548233032227
3100 time: 1.378829002380371
3200 time: 1.2134480476379395
3300 time: 1.1988680362701416
3400 time: 1.3604850769042969

In [278]:
map_center = [39.8283, -98.5795]
m = folium.Map(location=map_center, zoom_start=4)
for city_key in all_cities:
    for road in all_cities[city_key].roads:
        city1, city2 = road.connected_cities
        lat1, long1 = city1.lat_long
        lat2, long2 = city2.lat_long
        folium.PolyLine(locations=[[lat1, -long1], [lat2, -long2]]).add_to(m)
display(m)

In [272]:
for road in shortest_paths[('Los Angeles (34.02, 118.41)','New York (40.66, 73.94)')]:
    print(road)

(Dayton, Fishers)
(Corona, Menifee)
(Springfield, Tulsa)
(Downey, Fullerton)
(Allentown, Edison)
(Columbus, Pittsburgh)
(Downey, Inglewood)
(Allentown, Pittsburgh)
(Inglewood, Los Angeles)
(Springfield, St. Louis)
(Anaheim, Fullerton)
(Menifee, Phoenix)
(Elizabeth, New York)
(Amarillo, Oklahoma City)
(Fishers, Indianapolis)
(Elizabeth, Woodbridge)
(Albuquerque, Amarillo)
(Edison, Woodbridge)
(Columbus, Dayton)
(Indianapolis, St. Louis)
(Albuquerque, Phoenix)
(Anaheim, Corona)
(Oklahoma City, Tulsa)


In [269]:
all_cities

{'New York (40.66, 73.94)': <__main__.City at 0x168c417f0>,
 'Los Angeles (34.02, 118.41)': <__main__.City at 0x168c41dc0>,
 'Chicago (41.84, 87.68)': <__main__.City at 0x168c43b30>,
 'Houston (29.79, 95.39)': <__main__.City at 0x168c42b10>,
 'Phoenix (33.57, 112.09)': <__main__.City at 0x110087350>,
 'Philadelphia (40.01, 75.13)': <__main__.City at 0x11036fb00>,
 'San Antonio (29.46, 98.52)': <__main__.City at 0x11036fd40>,
 'San Diego (32.81, 117.14)': <__main__.City at 0x11036e1e0>,
 'Dallas (32.79, 96.77)': <__main__.City at 0x11036d8e0>,
 'Jacksonville (30.34, 81.66)': <__main__.City at 0x11036c3e0>,
 'Austin (30.3, 97.75)': <__main__.City at 0x11036c4a0>,
 'Fort Worth (32.78, 97.35)': <__main__.City at 0x11036c4d0>,
 'San Jose (37.3, 121.81)': <__main__.City at 0x147f43200>,
 'Columbus (39.99, 82.99)': <__main__.City at 0x147f43a10>,
 'Charlotte (35.21, 80.83)': <__main__.City at 0x147f43fe0>,
 'Indianapolis (39.78, 86.15)': <__main__.City at 0x147f43650>,
 'San Francisco (37.73,

In [274]:
roads_sorted_by_population[-1].connected_population

12078949